In [ ]:
import os
os.sys.path.insert(0, "..")

import arviz
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pystan
import seaborn as sns
import stravalib
from secret import *

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 7)

# GAP calculation

## 1. Get data from the strava API

In [ ]:
client = stravalib.Client()

response = client.exchange_code_for_token(
    CLIENT_ID,
    CLIENT_SECRET,
    CODE,
)
client.access_token = response["access_token"]

In [ ]:
%%capture
activities = client.get_activities()
runs = (ai for ai in activities if ai.type == "Run" and ai.manual == False)

speed = []
elevation = []
distance = []
date = []

for run in runs:
    x = client.get_activity(run.id)
    speed.append(x.average_speed)
    elevation.append(x.total_elevation_gain)
    distance.append(x.distance)
    date.append(x.start_date)

speed = np.array([s.num for s in speed])
elevation = np.array([e.num for e in elevation])
distance = np.array([d.num for d in distance])
date = np.array([date])

ind = distance > 5000.0
speed, elevation, distance, date = (
    speed[ind][::-1], 
    elevation[ind][::-1], 
    distance[ind][::-1], 
    date.ravel()[ind][::-1]
)

In [ ]:
pd.DataFrame(
    {
        "speed": speed,
        "elevation": elevation,
        "distance": distance,
        "date": date,
    }
).to_csv("strava_data.csv")

## 2. Plot the data

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))
plt.scatter(np.log(distance), np.log(speed), c=elevation)
plt.xlabel("log(distance)", fontsize=20)
plt.ylabel("log(speed) (m/s)", fontsize=20)
cb = plt.colorbar()
cb.set_label("elevation gain (m)", size=20)
plt.savefig("distance_vs_speed.png")

## 3. Simple statistical model

$$\log v\sim \mathcal{N}(\alpha + \beta_{\mathrm{elev}}\log (\mathrm{elev} + 10) + \beta_{\mathrm{dist}}\log (\mathrm{dist}),\, \sigma)$$

In [ ]:
stan_code = """
data {
  int n;
  vector[n] log_speed;
  vector[n] elevation_gain;
  vector[n] log_distance;
}
parameters {
  real beta_elevation;
  real beta_distance;
  real sigma;
}
transformed parameters {
  vector[n] z = beta_elevation * elevation_gain + beta_distance * log_distance;
}
model {
  beta_elevation ~ normal(0, 1);
  beta_distance ~ normal(0, 1);
  sigma ~ normal(0, 5);
  log_speed ~ normal(z, sigma);
}
generated quantities {
  vector[n] log_speed_rep;
  for (i in 1:n) {
    log_speed_rep[i] = normal_rng(z[i], sigma);
  }
}
"""

model = pystan.StanModel(model_code=stan_code)

In [ ]:
log_speed = np.log(speed)
whitened_log_speed = (log_speed - log_speed.mean()) / log_speed.std()
whitened_elevation_gain = (np.log(elevation + 10) - np.log(elevation + 10).mean()) / np.log(elevation + 10).std()
whitened_distance = (np.log(distance) - np.log(distance).mean()) / np.log(distance).std()

stan_data = {
    "n": len(speed),
    "log_speed": whitened_log_speed,
    "elevation_gain": whitened_elevation_gain,
    "log_distance": whitened_distance
}

fit = model.sampling(stan_data)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
sns.distplot(fit["beta_distance"], ax=ax[0], kde=False, color="r")
sns.distplot(fit["beta_elevation"], ax=ax[1], kde=False, color="b")
ax[0].set_xlabel(r"$\beta_\mathrm{distance}$", fontsize=20)
ax[1].set_xlabel(r"$\beta_\mathrm{elevation}$", fontsize=20)
for axi in ax:
    axi.yaxis.set_visible(False)
    
plt.tight_layout()
plt.savefig("model_params.png")

In [ ]:
dfs = []
for i in range(len(speed)):
    dfs.append(pd.DataFrame(
        {"x": np.tile(i, 4000), 
         "y": (
             whitened_log_speed[i] - fit["log_speed_rep"][:, i]
         )
        }
    ))

sns.violinplot(x="x", y="y", data=pd.concat(dfs))
plt.gca().xaxis.set_visible(False)
plt.ylabel(r"$\log(\mathrm{speed}) - \log(\mathrm{speed})_\mathrm{rep}$", size=20)
plt.axhline(1.0, ls="--", c="r")
plt.axhline(-1.0, ls="--", c="r")
plt.axhline(2.0, ls="--", c="r")
plt.axhline(-2.0, ls="--", c="r")
plt.savefig("residuals_simple.png")

## 4. Calculate "GAP"

Calculate GAP as 

$$\mathbb{E}[\log v - \beta_\mathrm{elev}\mathrm{elev}]$$

In [ ]:
def transform_elevation(g):
    return (np.log(g + 10) - np.log(elevation + 10).mean()) / np.log(elevation + 10).std()

def sample_gap(v, g):
    return v + fit["beta_elevation"] * (transform_elevation(0.0) - g)

def mean_gap(v, g):
    return sample_gap(v, g).mean()

def inverse_transform_gap(gap):
    return np.exp(gap * np.log(speed).std() + np.log(speed).mean())

def ms_to_minkm(speed):
    return 1000 / 60 / speed

gap = inverse_transform_gap(
    np.array([sample_gap(vi, gi) for vi, gi in zip(whitened_log_speed, whitened_elevation_gain)])
)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

colors = np.log(elevation)
colors -= colors.min()
colors *=  (1./colors.max())

scatter_plot = plt.scatter(
    ms_to_minkm(speed), 
    ms_to_minkm(gap).mean(axis=1), 
    c=elevation,
    cmap="plasma",
    edgecolors="k"
)
cb = plt.colorbar(scatter_plot)
cb.set_label(label="elevation gain (m)",size=20)

_, __, errorlinecollection = plt.errorbar(
    ms_to_minkm(speed), 
    ms_to_minkm(gap).mean(axis=1), 
    yerr=2 * ms_to_minkm(gap).std(axis=1),
    marker="",
    ls="none",
    zorder=0,
    ecolor="0.5",
    alpha=0.5,
    capsize=5
)

ax.set_ylabel("GAP (minutes per km)", size=20)
ax.set_xlabel("pace (minutes per km)", size=20);
plt.plot(
    [ms_to_minkm(speed).min(), ms_to_minkm(speed).max()], 
    [ms_to_minkm(speed).min(), ms_to_minkm(speed).max()],
    zorder=0,
    ls="--",
    c="0.6"
)

plt.tight_layout()
plt.savefig("gap_vs_true.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ind = elevation.argmax()
sns.distplot(ms_to_minkm(gap[ind]), kde=False)
plt.axvline(4.78, ls="--", label="strava GAP", c="k")
plt.axvline(ms_to_minkm(speed[ind]), label="actual average pace", c="0.3", ls="-.")
plt.xlabel("GAP (minutes per km)", fontsize=20)
plt.legend(loc="upper left", fontsize=15)
ax.yaxis.set_visible(False)
plt.savefig("gap_vs_strava.png")

Issue: elevation gain just says how much you went up, not how much you came down! But this is probably fine since nearly 100% of my runs are loops, so it is given that I go up as much as I come down.

In [ ]:
plt.scatter(elevation, distance / 1000, c=1000 / (60 * speed))
plt.xlabel("elevation gain (m)", fontsize=20)
plt.ylabel("distance (km)", fontsize=20)
plt.tight_layout()
plt.ylim((2.0, 22.0))
cb = plt.colorbar()
cb.set_label(label="pace (min per km)", size=20)
plt.savefig("distance_vs_elevation.png")

## Inferring fitness over time

In [ ]:
hierarchical_stan_code = """
data {
  int n;
  vector[n] log_speed;
  vector[n] elevation_gain;
  vector[n] log_distance;
}
parameters {
  real beta_elevation;
  real beta_distance;
  real<lower=0> sigma;
  vector[n] fitness_std;
  real<lower=0> sigma_rw;
}
transformed parameters {
  vector[n] fitness;
  vector[n] z;
  fitness[1] = fitness_std[1];
  for (i in 2:n) {
    fitness[i] = fitness_std[i] * sigma_rw + fitness[i - 1];
  }
  
  z = fitness + beta_elevation * elevation_gain + beta_distance * log_distance;
}
model {
  beta_elevation ~ normal(0, 1);
  beta_distance ~ normal(0, 1);
  sigma ~ normal(0, 1);
  log_speed ~ normal(z, sigma);
  fitness_std ~ normal(0, 1);
  sigma_rw ~ normal(0, 1);
}
generated quantities {
  vector[n] log_speed_rep;
  for (i in 1:n) {
    log_speed_rep[i] = normal_rng(z[i], sigma);
  }
}
"""

hierarchical_model = pystan.StanModel(model_code=hierarchical_stan_code)

In [ ]:
fit_hierarchical = hierarchical_model.sampling(data=stan_data, control={"adapt_delta": 0.99}, iter=8000, warmup=7000)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
pd_date = pd.Series(date)
ax.fill_between(
    pd_date,
    np.percentile(fit_hierarchical["fitness"], 16, axis=0),
    np.percentile(fit_hierarchical["fitness"], 84, axis=0),
    color="0.7"
)
ax.plot_date(
    pd_date,
    np.percentile(fit_hierarchical["fitness"], 50, axis=0),
    c="k"
)
ax.set_ylabel("Fitness", size=20)
ax.set_xlabel("Date", size=20)
ax.axvline("2019-05-19", ls="--", c="r", label="half marathon pb (1)", lw=3)
ax.axvline("2019-06-08", ls="--", c="k", label="half marathon pb (2)", lw=3)
ax.axvline("2018-09-09", ls="--", c="y", label="injury", lw=3)
ax.axvline("2018-07-14", ls="--", c="g", label="5km pb", lw=3)
ax.legend(prop={"size": 15})
fig.tight_layout()
plt.savefig("fitness_trend.png")

## Posterior predictive checks

In [ ]:
dfs = []
for i in range(len(speed)):
    dfs.append(pd.DataFrame(
        {"x": np.tile(i, 4000), 
         "y": (
             whitened_log_speed[i] - fit_hierarchical["log_speed_rep"][:, i]
         )
        }
    ))

sns.violinplot(x="x", y="y", data=pd.concat(dfs))
plt.gca().xaxis.set_visible(False)
plt.ylabel(r"$\log(\mathrm{speed}) - \log(\mathrm{speed})_\mathrm{rep}$", size=20)
plt.axhline(1.0, ls="--", c="r")
plt.axhline(-1.0, ls="--", c="r")
plt.axhline(2.0, ls="--", c="r")
plt.axhline(-2.0, ls="--", c="r")
plt.savefig("residuals.png")